# Singular Value Decomposition on Images: Dropping Singular Values

In the previous notebook, we demonstrated SVD on a small numeric matrix.  
Now, let’s see how dropping singular values affects a **real grayscale image**.

This connects directly to image compression and de-enhancement: by discarding smaller singular values, we lose fine details but retain the overall structure of the image.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import data

# Load a sample grayscale image (512x512 cameraman)
image = data.camera()

plt.imshow(image, cmap='gray')
plt.title("Original Image")
plt.axis('off')
plt.show()

print("Image shape:", image.shape)

## Step 1: Perform SVD

We’ll decompose the image matrix into U, Σ, and Vᵀ.

In [ ]:
U, S, VT = np.linalg.svd(image, full_matrices=False)

print("Number of singular values:", len(S))

## Step 2: Visualize Singular Values

As before, we plot the singular values to see their relative importance.  
Notice how the first few are much larger — these capture most of the image’s structure.

In [ ]:
plt.figure(figsize=(8, 4))
plt.semilogy(S, marker='o')  # log scale to better see the decay
plt.title("Singular Values of the Image")
plt.xlabel("Index")
plt.ylabel("Singular Value (log scale)")
plt.grid(True)
plt.show()

## Step 3: Reconstruct with Fewer Singular Values

We’ll reconstruct the image using:
- **k = 512 (all values)** → perfect reconstruction  
- **k = 50** → high compression, still recognizable  
- **k = 10** → only the roughest structure remains  
- **k = 5** → image becomes very blurry  

In [ ]:
def reconstruct(U, S, VT, k):
    U_k = U[:, :k]
    S_k = np.diag(S[:k])
    VT_k = VT[:k, :]
    return U_k @ S_k @ VT_k

# Try different levels of compression
k_values = [512, 50, 10, 5]
reconstructions = [reconstruct(U, S, VT, k) for k in k_values]

## Step 4: Compare Reconstructions

In [ ]:
fig, axs = plt.subplots(1, len(k_values)+1, figsize=(18, 6))

axs[0].imshow(image, cmap='gray')
axs[0].set_title("Original")
axs[0].axis('off')

for i, (k, recon) in enumerate(zip(k_values, reconstructions)):
    axs[i+1].imshow(recon, cmap='gray')
    axs[i+1].set_title(f"k={k}")
    axs[i+1].axis('off')

plt.show()


## Observations

- **k = all values (512):** Perfect reconstruction — identical to the original.  
- **k = 50:** Clear image, only fine textures missing.  
- **k = 10:** Still recognizable, but blurry and lacking detail.  
- **k = 5:** Most detail lost, only the rough outline remains.  

This demonstrates how SVD enables a trade-off between **file size (compression)** and **image clarity (detail retention)**.